In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, log

Results of the experiment from Ariely (2008):

|     | Experiment 1            | Experiment 2           |     |
| --- | ---                     | ---                    | --- |
|16   | Web only @ \$59         | Web only @ \$59        | 68  |
|0    | Print only @ \$125      |                        |     |
|84   | Print and web @ \$125   | Print and web @ \$125  | 32  |

In [2]:
data = pd.DataFrame({'Experiment': [1, 2], 
                     'Web_only': [16, 68], 
                     'Print_and_web': [84, 32]})


In [3]:
data

,Experiment,Web_only,Print_and_web
0,1,16,84
1,2,68,32


In [4]:
database = db.Database('data', data)
globals().update(database.variables)

# Logit model

## Parameters 

We estimate two parameters:
- a coefficient for the cost,
- a constant associated with alternatives involving the web option.

In [5]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
cte_web = Beta('cte_web', 0, None, None, 0)

## Utility functions

In [6]:
V_web = beta_cost * 59 + cte_web
V_print = beta_cost * 125
V_print_web = beta_cost * 125 + cte_web

In [7]:
V = {
    1: V_web, 
    2: V_print, 
    3: V_print_web
}

## Availability conditions

Alternative 2 (print only) is only available in experiment 1.

In [8]:
av = {1: 1, 2: Experiment == 1, 3: 1}

## Log likelihood

As alternative 2 is never chosen, it does not appear in the log likelihood function.

In [9]:
loglike = (
    Web_only * models.loglogit(V, av, 1) + 
    Print_and_web * models.loglogit(V, av, 3)
)

In [10]:
biogeme = bio.BIOGEME(database, loglike)
biogeme.modelName = 'logit'

In [11]:
results = biogeme.estimate()

In [12]:
print(results.printGeneralStatistics())

Number of estimated parameters:	2
Sample size:	2
Excluded observations:	0
Init log likelihood:	-179.1759
Final log likelihood:	-136.0584
Likelihood ratio test for the init. model:	86.23503
Rho-square for the init. model:	0.241
Rho-square-bar for the init. model:	0.229
Akaike Information Criterion:	276.1169
Bayesian Information Criterion:	273.5032
Final gradient norm:	9.5031E-04
Nbr of threads:	16



In [13]:
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,0.004891,0.002171,2.252949,0.024262,0.011435,0.427678,0.668886
cte_web,14.341971,170.846325,0.083947,0.933099,1.244489,11.524387,0.000000


We note that the cost coefficient is not significantly different from zero.

# Logit model with a decoy

We introduce now a variable that identifies the presence of the decoy. The behavioral assumption is that the "print and web" alternative becomes attractive because customers perceive that they obtain a significant discount, as they can obtain this alternative for the same price as "print only". Therefore, the presence of the dominated alternative "print only" (in experiment 1) increases the perceived utility of the alternative "print and web".

In [14]:
decoy = Experiment == 1
beta_decoy = Beta('beta_decoy', 0, None, None, 0)

In [15]:
V_print_web_decoy = (
    beta_cost * 125 +  
    cte_web +
    beta_decoy * decoy
)

In [16]:
V_decoy = {
    1: V_web, 
    2: V_print, 
    3: V_print_web_decoy
}

In [17]:
loglike_decoy = (
    Web_only * models.loglogit(V_decoy, av, 1) + 
    Print_and_web * models.loglogit(V_decoy, av, 3)
)

In [18]:
biogeme_decoy = bio.BIOGEME(database, loglike_decoy)
biogeme.modelName = 'logit_decoy'

In [19]:
results_decoy = biogeme_decoy.estimate()

In [20]:
print(results_decoy.printGeneralStatistics())

Number of estimated parameters:	3
Sample size:	2
Excluded observations:	0
Init log likelihood:	-179.1759
Final log likelihood:	-106.654
Likelihood ratio test for the init. model:	145.0439
Rho-square for the init. model:	0.405
Rho-square-bar for the init. model:	0.388
Akaike Information Criterion:	219.308
Bayesian Information Criterion:	215.3874
Final gradient norm:	6.9715E-04
Nbr of threads:	16



In [21]:
results_decoy.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.011421,0.003248,-3.516166,4.378276e-04,5.189554e-13,-2.200726e+10,0.0
beta_decoy,2.412000,0.346930,6.952403,3.591127e-12,4.843616e-11,4.979750e+10,0.0
cte_web,11.652095,123.549649,0.094311,9.248621e-01,1.000001e+00,1.165209e+01,0.0


Now, the cost coefficient is negative, and significantly different from zero. And the impact of the decoy is to increase the utility of the "print and web" alternative, as expected. 

In this example, we have been able to capture the apparent irrationality reported by Ariely (2008) with a simple specification of the utility functions, resulting from the behavioral assumption that customers are attracted by alternatives associated with a discount. More complex examples may involve subjectivity, perception, and attitudes to address the apparent irrationality. These behavioral concepts cannot easily be captured by observed variables. But, as we will see later, they can be captured by so-called "latent variables", that can be included in the choice models. 